In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/class-1/class.csv
/kaggle/input/sf-review-stars-prediction/submission.csv
/kaggle/input/sf-review-stars-prediction/baseline.ipynb
/kaggle/input/sf-review-stars-prediction/test.csv
/kaggle/input/sf-review-stars-prediction/train.csv


In [2]:
pip install ktrain

     |████████████████████████████████| 25.2MB 561kB/s 
     |████████████████████████████████| 10.0MB 35.6MB/s 
     |████████████████████████████████| 245kB 29.9MB/s 
     |████████████████████████████████| 3.1MB 36.5MB/s 
     |████████████████████████████████| 573kB 37.4MB/s 
     |████████████████████████████████| 3.7MB 32.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.13.2-cp36-none-any.whl size=25239774 sha256=e5ee83cf720ad2abbf77219c54bd78cf3060fcd61a0e626306f607f3a5fa6144
  Stored in directory: /root/.cache/pip/wheels/95/fb/62/cf5424c7a9c267b78db4efacfe8b4c3a0a3f1a755f2d63e428
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=6d3ffc1ee1c538b8a6244b62be8e8cf3671c7f6be98c7491edc03ff0ef5e5aba
  Stored in directory: /root/.cache/pip/wheels/bd/27/da/ffc2d573aa48b87440ec4f98bc7c992e3a2d899edb2d22ef9e
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=45c0c19f6930d3a85ba61865f3e8d103c0a95c5b49f9e3

In [3]:
import ktrain
from ktrain import text

In [4]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [5]:
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from ast import literal_eval
import torch
from sklearn.cluster import DBSCAN
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.manifold import TSNE

import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print("We're using TF", tf.__version__)
import keras
import keras.models as M
import keras.layers as L
import keras.backend as K
from tensorflow.keras import backend as K
from tensorflow.keras import layers as L
from tensorflow.keras import models as M
import matplotlib.pyplot as plt

submission = pd.read_csv('/kaggle/input/sf-review-stars-prediction/submission.csv')
df_train = pd.read_csv('/kaggle/input/sf-review-stars-prediction/train.csv', parse_dates = True)
df_test = pd.read_csv('/kaggle/input/sf-review-stars-prediction/test.csv', parse_dates = True)
df_test.drop('index', axis = 1, inplace = True)
df_train['sample'] = 1
df_test['sample'] = 0
df_test['overall'] = 0
data = df_test.append(df_train, sort = False).reset_index(drop = True)
data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'ID'], axis = 1, inplace = True)
data['reviewTime'] = pd.to_datetime(data['reviewTime'])

We're using TF 2.1.0


Using TensorFlow backend.


In [6]:
data = data.drop([ 'asin',        'helpful', 'reviewTime',
           'reviewerID',   'reviewerName',        'summary', 'unixReviewTime',
             'features',], axis = 1)

In [7]:
train_data = data.query('sample == 1').drop('sample', axis = 1)
classes = ['overall_1.0', 'overall_2.0', 'overall_3.0', 'overall_4.0', 'overall_5.0']
train_data = pd.get_dummies(train_data, columns=['overall'])

In [8]:
train_data

,reviewText,overall_1.0,overall_2.0,overall_3.0,overall_4.0,overall_5.0
1000,I enjoy vintage books and movies so I enjoyed ...,0,0,0,0,1
1001,This book is a reissue of an old one; the auth...,0,0,0,1,0
1002,This was a fairly interesting read. It had ol...,0,0,0,1,0
1003,I'd never read any of the Amy Brewster mysteri...,0,0,0,0,1
1004,"If you like period pieces - clothing, lingo, y...",0,0,0,1,0
...,...,...,...,...,...,...
1995,I enjoyed reading this story. The characters ...,0,0,0,0,1
1996,This was a fun book to read for the 2nd time. ...,0,0,0,1,0
1997,Or just a Christmas romance to make you crack-...,0,0,0,0,1
1998,"I read ""Stop the wedding"" by this author and w...",1,0,0,0,0


In [9]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(train_df=train_data, 
                                                                   text_column='reviewText',
                                                                   label_columns=classes,
                                                                   val_pct=0.15, 
                                                                   max_features=20000, 
                                                                   maxlen=200,
                                                                   preprocess_mode='bert',
                                                                   ngram_range=1)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [10]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 200
done.


In [11]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test))

In [12]:
learner.autofit(1e-5, 6)



begin training using triangular learning rate policy with max lr of 1e-05...
Train on 850 samples, validate on 150 samples
Epoch 1/6
850/850 [==============================] - 31s 37ms/sample - loss: 1.2998 - acc: 0.4753 - val_loss: 1.2311 - val_acc: 0.5467
Epoch 2/6
850/850 [==============================] - 21s 24ms/sample - loss: 1.1865 - acc: 0.5000 - val_loss: 1.1464 - val_acc: 0.5733
Epoch 3/6
850/850 [==============================] - 21s 24ms/sample - loss: 1.1051 - acc: 0.5212 - val_loss: 1.0837 - val_acc: 0.6000
Epoch 4/6
850/850 [==============================] - 21s 24ms/sample - loss: 1.0333 - acc: 0.5659 - val_loss: 1.0832 - val_acc: 0.5867
Epoch 5/6
850/850 [==============================] - 21s 24ms/sample - loss: 0.9395 - acc: 0.6153 - val_loss: 1.0250 - val_acc: 0.6067
Epoch 6/6
850/850 [==============================] - 21s 24ms/sample - loss: 0.8558 - acc: 0.6494 - val_loss: 1.0090 - val_acc: 0.5733


In [13]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [14]:
test_data = data.query('sample == 0').drop(['sample', 'overall'], axis = 1)

In [15]:
predict = predictor.predict(test_data['reviewText'].values)

In [16]:
test_df = pd.read_csv('/kaggle/input/sf-review-stars-prediction/test.csv')
submission = pd.DataFrame({
    'ID': test_df.ID,
    'overall': predict
})
submission['overall'] = submission['overall'].apply(lambda x: x[-3:])
submission.to_csv('submission.csv', index=None)
submission.overall.value_counts()

5.0    726
4.0    185
3.0     87
2.0      2
Name: overall, dtype: int64